In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
import sys
import datetime as datetime
from datetime import datetime as dt
from datetime import timedelta as td
from datetime import date as dater
df = pd.read_csv(r'C:/Users/enoch/Desktop/MLvsCOVID/completedataset.csv')
df=df.set_index(['county','date'])
df=df.loc['Los Angeles']
df

,retailandrec,groceryandpharm,parks,transit,work,residential,confirmed_cases,deaths,new_confirmed_cases,new_deaths
date,,,,,,,,,,
2020-02-15,1.0,0.0,13.0,-1.0,-1.0,0.0,1,0,0.0,0.0
2020-02-16,4.0,-1.0,27.0,-1.0,0.0,-1.0,1,0,0.0,0.0
2020-02-17,7.0,0.0,30.0,-11.0,-32.0,7.0,1,0,0.0,0.0
2020-02-18,-1.0,0.0,11.0,2.0,2.0,0.0,1,0,0.0,0.0
2020-02-19,-1.0,-1.0,13.0,1.0,2.0,0.0,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-07-06,-29.0,-11.0,-6.0,-40.0,-42.0,14.0,116670,3537,1646.0,50.0
2020-07-07,-29.0,-9.0,-3.0,-38.0,-42.0,15.0,120764,3582,4094.0,45.0
2020-07-08,-30.0,-10.0,-2.0,-39.0,-41.0,15.0,123256,3643,2492.0,61.0


In [16]:
def adddelaycolumn(numdays):
    relevantdate = df.index.tolist() 
    bchanges =[]

    for date in relevantdate:

        newdate = dt.strptime(date, '%Y-%m-%d')+td(days=numdays)
        #print (newdate)
        #print(str(newdate)[0:10])
        try:
            bchanges.append(df.loc[str(newdate)[0:10]].new_confirmed_cases)
        except:
            bchanges.append(np.nan)
    

    string= str(numdays)+'dayslaternewcases'
    #print(changes) 
    df[string]=bchanges

    df.corr()


adddelaycolumn(7)
df.corr()

,retailandrec,groceryandpharm,parks,transit,work,residential,confirmed_cases,deaths,new_confirmed_cases,new_deaths,7dayslaternewcases
retailandrec,1.000000,0.791983,0.747333,0.959503,0.850872,-0.865471,-0.179406,-0.245263,-0.314998,-0.568584,-0.353856
groceryandpharm,0.791983,1.000000,0.575564,0.782525,0.729465,-0.724027,-0.097843,-0.118637,-0.214326,-0.396751,-0.252947
parks,0.747333,0.575564,1.000000,0.749811,0.575231,-0.687126,0.241684,0.239590,0.087963,-0.215999,0.061567
transit,0.959503,0.782525,0.749811,1.000000,0.927973,-0.930551,-0.235715,-0.274067,-0.353377,-0.519303,-0.387479
work,0.850872,0.729465,0.575231,0.927973,1.000000,-0.964161,-0.259196,-0.290362,-0.382344,-0.537705,-0.404168
residential,-0.865471,-0.724027,-0.687126,-0.930551,-0.964161,1.000000,0.103562,0.139403,0.258629,0.508345,0.292034
confirmed_cases,-0.179406,-0.097843,0.241684,-0.235715,-0.259196,0.103562,1.000000,0.977156,0.882746,0.455020,0.867429
deaths,-0.245263,-0.118637,0.239590,-0.274067,-0.290362,0.139403,0.977156,1.000000,0.865896,0.527721,0.854369
new_confirmed_cases,-0.314998,-0.214326,0.087963,-0.353377,-0.382344,0.258629,0.882746,0.865896,1.000000,0.527256,0.824322
new_deaths,-0.568584,-0.396751,-0.215999,-0.519303,-0.537705,0.508345,0.455020,0.527721,0.527256,1.000000,0.520578


In [17]:
# Remove rows with missing target, separate target from predictors
needsprediction=df[df['7dayslaternewcases'].isnull()]
print(needsprediction)
noneed=df[df['7dayslaternewcases'].notnull()]
print(noneed)

            retailandrec  groceryandpharm  parks  transit  work  residential  \
date                                                                           
2020-07-04         -51.0             -9.0  -35.0    -44.0 -36.0         12.0   
2020-07-05         -40.0            -21.0  -24.0    -43.0 -25.0          9.0   
2020-07-06         -29.0            -11.0   -6.0    -40.0 -42.0         14.0   
2020-07-07         -29.0             -9.0   -3.0    -38.0 -42.0         15.0   
2020-07-08         -30.0            -10.0   -2.0    -39.0 -41.0         15.0   
2020-07-09         -29.0             -9.0   -5.0    -40.0 -41.0         16.0   
2020-07-10         -32.0             -8.0   -8.0    -40.0 -40.0         16.0   

            confirmed_cases  deaths  new_confirmed_cases  new_deaths  \
date                                                                   
2020-07-04           113622    3457               3187.0         0.0   
2020-07-05           115024    3487               1402.0       

In [18]:
X = noneed.drop(columns=['7dayslaternewcases'])
y=noneed['7dayslaternewcases']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

print(X)
print(y)

            retailandrec  groceryandpharm  parks  transit  work  residential  \
date                                                                           
2020-02-15           1.0              0.0   13.0     -1.0  -1.0          0.0   
2020-02-16           4.0             -1.0   27.0     -1.0   0.0         -1.0   
2020-02-17           7.0              0.0   30.0    -11.0 -32.0          7.0   
2020-02-18          -1.0              0.0   11.0      2.0   2.0          0.0   
2020-02-19          -1.0             -1.0   13.0      1.0   2.0          0.0   
...                  ...              ...    ...      ...   ...          ...   
2020-06-29         -28.0            -14.0   -5.0    -39.0 -39.0         14.0   
2020-06-30         -27.0            -11.0   -4.0    -39.0 -40.0         14.0   
2020-07-01         -26.0             -9.0   -4.0    -37.0 -40.0         14.0   
2020-07-02         -25.0             -3.0   -3.0    -36.0 -42.0         14.0   
2020-07-03         -27.0              5.

In [19]:
imputer = SimpleImputer()
# Your code here
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print(imputed_X_train)

     retailandrec  groceryandpharm  parks  transit  work  residential  \
0             4.0              6.0    9.0     -1.0   2.0          0.0   
1             2.0              1.0   13.0      3.0   3.0         -1.0   
2           -51.0            -19.0  -54.0    -52.0 -43.0         20.0   
3           -51.0            -27.0  -48.0    -54.0 -41.0         17.0   
4           -39.0             -6.0  -16.0    -43.0 -42.0         18.0   
..            ...              ...    ...      ...   ...          ...   
107           1.0             -1.0    8.0      2.0   4.0         -1.0   
108         -41.0            -12.0  -10.0    -43.0 -42.0         19.0   
109         -54.0            -20.0  -33.0    -51.0 -50.0         23.0   
110         -32.0             -8.0   -7.0    -39.0 -40.0         16.0   
111         -43.0            -19.0  -39.0    -52.0 -52.0         25.0   

     confirmed_cases  deaths  new_confirmed_cases  new_deaths  
0                7.0     0.0                  6.0         0

In [20]:
print("MAE (Imputation):")
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(imputed_X_train, y_train)
preds = model.predict(imputed_X_valid)
print(mean_absolute_error(y_valid, preds))

MAE (Imputation):
228.7275


In [23]:
print(needsprediction)
testing=needsprediction.drop(columns=['7dayslaternewcases'])
testing=pd.DataFrame(imputer.fit_transform(testing))
needsprediction['predictions']= model.predict(testing)



            retailandrec  groceryandpharm  parks  transit  work  residential  \
date                                                                           
2020-07-04         -51.0             -9.0  -35.0    -44.0 -36.0         12.0   
2020-07-05         -40.0            -21.0  -24.0    -43.0 -25.0          9.0   
2020-07-06         -29.0            -11.0   -6.0    -40.0 -42.0         14.0   
2020-07-07         -29.0             -9.0   -3.0    -38.0 -42.0         15.0   
2020-07-08         -30.0            -10.0   -2.0    -39.0 -41.0         15.0   
2020-07-09         -29.0             -9.0   -5.0    -40.0 -41.0         16.0   
2020-07-10         -32.0             -8.0   -8.0    -40.0 -40.0         16.0   

            confirmed_cases  deaths  new_confirmed_cases  new_deaths  \
date                                                                   
2020-07-04           113622    3457               3187.0         0.0   
2020-07-05           115024    3487               1402.0       

ValueError: Number of features of the model must match the input. Model n_features is 10 and input n_features is 11 